# IBAN stacker
This model is my first attempt to build a stacker to decide how/what/which IBAN is the true IBAN.


In [1]:
import ortec
import glob
import numpy as np
import pandas as pd
import keras

from keras.preprocessing.text import Tokenizer


/home/paperspace/anaconda3/envs/bletchley/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def pad(s, length=32):
    l = len(s)
    if l < length:
        o = s + ' ' * (max(0, (length-l)))
    elif l == length:
        o = s
    else:
        o = s[:length]
    return o

def apad(ss, length=32):
    """pad strings in list ss to required length
    set length to zero in order to pad to longest string found in s"""
    if length == 0:
        for s in ss:
            length = max(len(s), length)
    #
    print("padding strings to max length of {}".format(length))
    
    a = []
    for s in ss:
        l = len(s)

        if l < length:
            o = s + ' ' * (max(0, (length-l)))
        elif l == length:
            o = s
        else:
            o = s[:length]
        a.append(o)
        
#         p = ' ' * (max(0, (length-l)))
#         a.append(s + p)
    return a
#


In [3]:
xx =  ["abcd", "1234567"]
pp = apad(xx, length=6)
for x, p in zip(xx, pp):
    print(x, "_" +  p + "_") 
#

padding strings to max length of 6
abcd _abcd  _
1234567 _123456_


# load predictions and truth from disk
For now we load "fake truths" and "fake predictions" from generated data. Only IBANs in those, for now.



In [4]:
train_dir = "./train_G_iban/"
! ls -l {train_dir}

total 17880
-rw-rw-r-- 1 paperspace paperspace 3669791 Mar 16 14:47 iban_predicted_GX1_47k.csv
-rw-rw-r-- 1 paperspace paperspace 3858155 Mar 16 14:47 iban_predicted_GX2_47k.csv
-rw-rw-r-- 1 paperspace paperspace 3393175 Mar 16 14:47 iban_predicted_GX3_47k.csv
-rw-rw-r-- 1 paperspace paperspace 3808344 Mar 16 14:47 iban_predicted_GX4_47k.csv
-rw-rw-r-- 1 paperspace paperspace 3572045 Mar 16 14:47 iban_truth_GY_47k.csv


In [5]:
# filenames_all = glob.glob(train_dir + "iban_predicted*.csv")
# print("{} files found in directory".format(len(filenames_all)))

# batch_size = 32


In [6]:
! head {train_dir}/iban_predicted_GX1_47k.csv

uuid,x_name,x_kvk,x_iban,x_reference,x_total
a6ec0d75-f2d3-4965-b626-fca8fe70c286,NaN,NaN,XNL80 CObA 0969 0F380Z 5,NaN,NaN
9ababcab-b944-4c15-a192-a57ee63a5ebd,NaN,NaN,NL32 BcITP 8598 142O 38H,NaN,NaN
eb0715a4-d68f-4ac4-a6ab-429afb27cb24,NaN,NaN,NL470DNIB 3QI721 8240 06,NaN,NaN
d71c3ace-7083-4c31-a52e-d4ae0afba8f7,NaN,NaN,NLI4 RB4R 81j08 2396 61A,NaN,NaN
1e310847-f7e4-40f8-9d58-2441fc585902,NaN,NaN,NML19 BOFLAa 7381591 180,NaN,NaN
bf1593f4-b140-4d6c-8ee4-de560f41ba07,NaN,NaN,NL41 LOdJP 974382 099 45,NaN,NaN
257fe217-f13a-4787-9263-3a83a3269a91,NaN,NaN,nL131Q EBTUR661 1802 F76,NaN,NaN
ef1de86f-0446-455a-9f84-3d15e39a0fd6,NaN,NaN,NLT07 LOD K1762G 835141J,NaN,NaN
2ad4bf2b-080f-4b4f-bbe5-48eb64a384fd,NaN,NaN,JTNL43 cIC4 5033 715 538,NaN,NaN


In [7]:
%whos


Variable    Type        Data/Info
---------------------------------
Tokenizer   type        <class 'keras.preprocessing.text.Tokenizer'>
apad        function    <function apad at 0x7fe73c916f28>
glob        module      <module 'glob' from '/hom<...>y/lib/python3.6/glob.py'>
keras       module      <module 'keras' from '/ho<...>kages/keras/__init__.py'>
np          module      <module 'numpy' from '/ho<...>kages/numpy/__init__.py'>
ortec       module      <module 'ortec' from '/ho<...>ice_classifier/ortec.py'>
p           str         123456
pad         function    <function pad at 0x7fe7a41ff9d8>
pd          module      <module 'pandas' from '/h<...>ages/pandas/__init__.py'>
pp          list        n=2
train_dir   str         ./train_G_iban/
x           str         1234567
xx          list        n=2


In [8]:
preds_GX1 = pd.read_csv(train_dir + "iban_predicted_GX1_47k.csv")
preds_GX2 = pd.read_csv(train_dir + "iban_predicted_GX2_47k.csv")
preds_GX3 = pd.read_csv(train_dir + "iban_predicted_GX3_47k.csv")
preds_GX4 = pd.read_csv(train_dir + "iban_predicted_GX4_47k.csv")

truths_GY = pd.read_csv(train_dir + "iban_truth_GY_47k.csv")


In [9]:
truths_GY.head()

,uuid,y_name,y_kvk,y_iban,y_reference,y_total
0,a6ec0d75-f2d3-4965-b626-fca8fe70c286,NaN,NaN,NL80 COBA 0969 0380 25,NaN,NaN
1,9ababcab-b944-4c15-a192-a57ee63a5ebd,NaN,NaN,NL32 BCIT 8598 1422 38,NaN,NaN
2,eb0715a4-d68f-4ac4-a6ab-429afb27cb24,NaN,NaN,NL47 DNIB 3721 8240 06,NaN,NaN
3,d71c3ace-7083-4c31-a52e-d4ae0afba8f7,NaN,NaN,NL04 RBRB 8108 2396 61,NaN,NaN
4,1e310847-f7e4-40f8-9d58-2441fc585902,NaN,NaN,NL19 BOFA 7398 1591 18,NaN,NaN


In [10]:
train1 = preds_GX1['x_iban']
train2 = preds_GX2['x_iban']
train3 = preds_GX3['x_iban']
train4 = preds_GX4['x_iban']
print(train2.shape)
print(train2[1])

(47000,)
N L32   BMC T 858	  1422M 38


In [11]:
str_y_train = truths_GY['y_iban']
print(str_y_train.shape)
print(str_y_train[1])

(47000,)
NL32 BCIT 8598 1422 38


In [12]:
train1 = apad(train1, length=0)
train2 = apad(train2, length=0)
train3 = apad(train3, length=0)
train4 = apad(train4, length=0)
str_y_train = apad(str_y_train, length=0)

padding strings to max length of 26
padding strings to max length of 30
padding strings to max length of 21
padding strings to max length of 26
padding strings to max length of 22


In [13]:
print(str_y_train[1])

NL32 BCIT 8598 1422 38


In [14]:
cat_train = np.concatenate(
    (train1, 
     train2, 
     train3, 
     train4,
     str_y_train) , axis=0)

# Create our tokenizer
# We will tokenize on character level!
# We will NOT remove any characters

# CONSIDER:
# reducing num_words
# reduce padding length (change function to remove whitespace perhaps?)
# use filters

tokenizer = Tokenizer(num_words=48, char_level=True, filters=None, lower=False)
tokenizer.fit_on_texts(cat_train)
del cat_train


In [15]:
print("_"+train1[1]+"_")
print(len(train1[1]))

_NL32 BcITP 8598 142O 38H  _
26


In [300]:
m = len(train1)

In [301]:
X = []

# fold padded IBANs from 4 different sources together into one long string, for all rows in the array
# let's onehot the long (128 chars) string while we are iterating anyways
for i in range(m):
# for i in range(1):
    four_ibans = train1[i] + train2[i] + train3[i] + train4[i]

#    ###   FIXME FIXME FIXME  ###############################################
#     four_ibans = str_y_train[i] + str_y_train[i] + str_y_train[i] + str_y_train[i]  # BIG FAT HACK!
#    ###   FIXME FIXME FIXME  ###############################################

    onehot_ibans = tokenizer.texts_to_matrix(four_ibans)
    X.append(onehot_ibans)   # .flatten()
#
X_train = np.array(X)
print(X_train.shape)
print(X_train[1])

(47000, 103, 48)
[[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [19]:
tokenizer.word_counts

OrderedDict([('X', 13142),
             ('N', 304437),
             ('L', 261361),
             ('8', 271549),
             ('0', 263383),
             (' ', 1329175),
             ('C', 52886),
             ('O', 53399),
             ('b', 3870),
             ('A', 96095),
             ('9', 267254),
             ('6', 270936),
             ('F', 31424),
             ('3', 273233),
             ('Z', 9689),
             ('5', 268413),
             ('2', 272345),
             ('B', 152112),
             ('c', 1635),
             ('I', 46839),
             ('T', 34834),
             ('P', 16122),
             ('1', 270692),
             ('4', 270924),
             ('H', 40265),
             ('7', 271253),
             ('D', 40507),
             ('Q', 9912),
             ('R', 41174),
             ('j', 518),
             ('M', 22170),
             ('a', 2616),
             ('d', 1331),
             ('J', 6740),
             ('n', 7420),
             ('E', 31492),
             ('U', 2871

In [16]:
%whos

Variable      Type         Data/Info
------------------------------------
Tokenizer     type         <class 'keras.preprocessing.text.Tokenizer'>
apad          function     <function apad at 0x7fe73c916f28>
glob          module       <module 'glob' from '/hom<...>y/lib/python3.6/glob.py'>
keras         module       <module 'keras' from '/ho<...>kages/keras/__init__.py'>
np            module       <module 'numpy' from '/ho<...>kages/numpy/__init__.py'>
ortec         module       <module 'ortec' from '/ho<...>ice_classifier/ortec.py'>
p             str          123456
pad           function     <function pad at 0x7fe7a41ff9d8>
pd            module       <module 'pandas' from '/h<...>ages/pandas/__init__.py'>
pp            list         n=2
preds_GX1     DataFrame                             <...>n[47000 rows x 6 columns]
preds_GX2     DataFrame                             <...>n[47000 rows x 6 columns]
preds_GX3     DataFrame                             <...>n[47000 rows x 6 columns]
pred

In [302]:
# CONSIDER:
# not flattening target

y = []
for i in range(m):
    onehot_y = tokenizer.texts_to_matrix(str_y_train[i])
    y.append(onehot_y.flatten())
#
y_train = np.array(y)
print(type(y_train))
print(y_train.shape)
print(y_train[1])

<class 'numpy.ndarray'>
(47000, 1056)
[0. 0. 1. ... 0. 0. 0.]


## Note
we are going to train on four IBANs, without the other fields. Extend this POC later!

## Model building

In [303]:
input_shape = X_train[0].shape
input_width = input_shape[0]
input_depth = input_shape[1]
input_size = input_width * input_depth

output_size = y_train.shape[1]

print(input_shape, input_size)
print("m=", m)
print(output_size)

(103, 48) 4944
m= 47000
1056


In [304]:
import keras.models as km # import Sequential
import keras.layers as kl # import SimpleRNN, Dense,Activation, Conv1D, MaxPool1D, Flatten
import keras.optimizers as ko

# no free lunch
It is hard to keep the model small enough to fit in memory. I keep getting OOMs in the terminal.
Even when Trainable Parameters == 400000.

Also: complicated! With all the 2D inputs caused by onehot encoders and long strings.

In [305]:
%xdel model
model = km.Sequential()

model.add(kl.Dense(320, name='A', input_shape=input_shape))
model.add(kl.Activation('relu', name='a'))

model.add(kl.Dense(48, name='B'))
model.add(kl.Activation('relu', name='b'))

model.add(kl.Flatten(name='F'))
model.add(kl.Dropout(rate=.4, name='Fd'))

model.add(kl.Dense(64, name='K'))
model.add(kl.Activation('relu', name='k'))

model.add(kl.Dense(32, name='L'))
model.add(kl.Activation('relu', name='l'))

model.add(kl.Dense(22, name='N'))
model.add(kl.Activation('relu', name='n'))

model.add(kl.Dense(output_size, name='O'))
model.add(kl.Activation('relu', name='o'))


In [306]:
# CONSIDER: tweak learning rate

In [307]:
## sparse_categorical_crossentropy is like categorical crossentropy but without converting targets to one hot
model.compile(loss='mse',optimizer=keras.optimizers.adam(lr=.1))   # , metrics=['acc']
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
A (Dense)                    (None, 103, 320)          15680     
_________________________________________________________________
a (Activation)               (None, 103, 320)          0         
_________________________________________________________________
B (Dense)                    (None, 103, 48)           15408     
_________________________________________________________________
b (Activation)               (None, 103, 48)           0         
_________________________________________________________________
F (Flatten)                  (None, 4944)              0         
_________________________________________________________________
Fd (Dropout)                 (None, 4944)              0         
_________________________________________________________________
K (Dense)                    (None, 64)                316480    
__________

```
succesful .fit with:
68,588 params, bs 32, m=100
dito, m=1000
dito 85,008 params
dito params: 193,248
dito m=47k
dito bs=64
dito 273,264
dito bs=128
----------
Total params: 273,264, m=47k, bs=128, 48 chars in tokenizer (lowercase), input 128x48, output 32x48
dito params: 274,320 (1 more layer)
dito bs=256
```

In [308]:
train_hist = model.fit(x=X_train, y=y_train, batch_size=256, epochs=1, validation_split=.2, verbose=True)


Train on 37600 samples, validate on 9400 samples
Epoch 1/1
37600/37600 [==============================] - 3s 92us/step - loss: 3.0142 - val_loss: 0.0161


In [72]:
model.save('IBAN_stacker_226k.h5')

In [309]:
i = 0

In [321]:
i += 1
iban_x = X_train[i:i+1]
iban_true = y_train[i:i+1]

iban_pred = model.predict(x=iban_x)

pred_str = ""
pred_sum = 0
for f in iban_pred.flatten():
    pred_str += str(int(f))
    pred_sum += int(f)
true_str = ""
true_sum = 0
for f in iban_true.flatten():
    true_str += str(int(f))
    true_sum += int(f)
#

print(pred_str)
print(pred_sum)
print(true_str)
print(true_sum)
print(str_y_train[i:i+1])


0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000000000000000000000000000000000000000000